In [19]:
from collections import deque
import numpy as np
import random


class SequentialOrganAllocationMDP:
    def __init__(self, initial_state, available_organs):
        """
        Initialize the sequential MDP for organ allocation.

        Parameters:
        - initial_state: List of dictionaries representing waitlist candidates.
        - available_organs: Dictionary with available organs by blood type.
        """
        self.initial_state = (
            tuple((r['id'], r['age'], r['MELD'], r['blood_type'], r['allocated']) for r in initial_state),
            tuple(available_organs.items())
        )
        self.value_table = {self.initial_state: 0}  # Value function initialized for the initial state
        self.policy = {self.initial_state: None}  # Policy initialized for the initial state

    def calculate_reward(self, recipient):
        """
        Calculate reward for successfully allocating an organ.

        Parameters:
        - recipient: Tuple representing the recipient receiving the organ.

        Returns:
        - Reward value based on MELD score and age.
        """
        _, age, meld, _, _ = recipient
        return 10 + (1 / (1 + meld)) * 10 - (age * 0.1)

    def generate_next_state(self, state, action):
        """
        Generate the next state based on the current state and action.

        Parameters:
        - state: Tuple (recipients, available_organs).
        - action: Tuple (recipient_id, organ_type).

        Returns:
        - next_state: Updated state after taking the action.
        - reward: Associated reward for the action.
        """
        recipients, available_organs = state
        recipients = list(recipients)
        available_organs = dict(available_organs)

        if action is None:
            return (tuple(recipients), tuple(available_organs.items())), 0

        recipient_id, organ_type = action
        recipient_idx = next((i for i, r in enumerate(recipients) if r[0] == recipient_id), None)

        if recipient_idx is not None and available_organs[organ_type] > 0:
            if random.random() < 0.7:  # 70% success
                recipient = recipients[recipient_idx]
                recipients[recipient_idx] = (recipient[0], recipient[1], recipient[2], recipient[3], 1)
                available_organs[organ_type] -= 1
                reward = self.calculate_reward(recipient)
                return (tuple(recipients), tuple(available_organs.items())), reward
            else:  # 30% failure (recipient dies, organ is removed)
                recipient = recipients.pop(recipient_idx)
                available_organs[organ_type] -= 1
                reward = -100  # Large penalty for death
                return (tuple(recipients), tuple(available_organs.items())), reward

        return (tuple(recipients), tuple(available_organs.items())), 0

    def value_iteration(self, gamma=0.9, epsilon=0.01):
        """
        Perform value iteration for all reachable states starting from the initial state.

        Parameters:
        - gamma: Discount factor (default 0.9).
        - epsilon: Convergence threshold (default 0.01).
        """
        states_to_explore = deque([self.initial_state])

        while True:  # Outer loop for global convergence
            delta = 0  # Track the largest change in value across all states
            new_states_to_explore = deque()  # To track states added during this iteration

            while states_to_explore:  # Inner loop for processing current states
                current_state = states_to_explore.popleft()
                old_value = self.value_table[current_state]
                max_value = float('-inf')
                best_action = None

                recipients, available_organs = current_state
                available_organs = dict(available_organs)
                for recipient in recipients:
                    if recipient[4] == 0:  # Not yet allocated
                        for organ_type in available_organs.keys():
                            if available_organs[organ_type] > 0:
                                action = (recipient[0], organ_type)
                                next_state, reward = self.generate_next_state(current_state, action)
                                value = reward + gamma * self.value_table.get(next_state, 0)

                                if value > max_value:
                                    max_value = value
                                    best_action = action

                                if next_state not in self.value_table:
                                    self.value_table[next_state] = 0
                                    self.policy[next_state] = None
                                    new_states_to_explore.append(next_state)

                self.value_table[current_state] = max_value
                self.policy[current_state] = best_action
                delta = max(delta, abs(old_value - max_value))

            states_to_explore = new_states_to_explore  # Add newly discovered states

            if delta < epsilon and not states_to_explore:
                break  # Stop when values converge and no new states to explore

    def simulate_with_policy(self, steps=10):
        """
        Simulate the allocation process using the computed policy.

        Parameters:
        - steps: Number of allocation steps to simulate (default 10).

        Returns:
        - Total reward, total deaths, total allocations.
        """
        current_state = self.initial_state
        total_reward = 0
        total_deaths = 0
        total_allocations = 0

        for _ in range(steps):
            action = self.policy.get(current_state, None)
            if action is None:
                break

            next_state, reward = self.generate_next_state(current_state, action)

            if reward == -100:  # Death penalty
                total_deaths += 1
            elif reward > 0:  # Successful allocation
                total_allocations += 1

            total_reward += reward
            current_state = next_state

        return total_reward, total_deaths, total_allocations


# Reinitialize the example with a reduced state space for debugging
initial_state = [
    {'id': i, 'age': random.randint(20, 70), 'MELD': random.randint(10, 40), 'blood_type': random.choice(['A', 'B', 'O', 'AB']), 'allocated': 0}
    for i in range(1, 6)
]
available_organs = {'A': 0, 'B': 1, 'O': 1, 'AB': 1}

# Initialize and execute the MDP
mdp_model = SequentialOrganAllocationMDP(initial_state, available_organs)
mdp_model.value_iteration()
total_reward, total_deaths, total_allocations = mdp_model.simulate_with_policy(steps=10)

# Output the total reward, total deaths, and total allocations
total_reward, total_deaths, total_allocations


(18.51204481792717, 0, 3)